In [ ]:
pip install rasterio

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling

# Directory containing the input TIFF files
input_dir = "/content/drive/MyDrive/Heatstress/2023_0_1"
output_tiff = '/content/drive/MyDrive/output_count_40.tif'

# Load all tiff file paths
tiff_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.tif')])

# Open first file to get metadata (assuming all TIFFs have the same metadata)
with rasterio.open(tiff_files[0]) as src:
    meta = src.meta.copy()
    nodata_value = src.nodata

# Initialize an array to store the counts with same shape as each input TIFF
output_array = np.full((meta['height'], meta['width']), nodata_value, dtype=np.int32)

# Process each pixel for consecutive streaks of "1"
for row in range(meta['height']):
    for col in range(meta['width']):
        consecutive_count = 0
        max_streak = 0

        for tiff_path in tiff_files:
            with rasterio.open(tiff_path) as src:
                pixel_value = src.read(1, resampling=Resampling.nearest)[row, col]

                # Check if pixel is NoData; if so, break and mark as NoData
                if pixel_value == nodata_value:
                    max_streak = nodata_value
                    break

                # Count consecutive "1"s
                if pixel_value == 1:
                    consecutive_count += 1
                else:
                    if consecutive_count >= 40:  # Keep count only if 3 or more consecutive "1"s
                        max_streak = consecutive_count
                    consecutive_count = 0  # Reset streak on "0"

        # Update output array for pixels with streak >= 3, else set to 0 or NoData
        if max_streak != nodata_value:
            output_array[row, col] = max_streak if max_streak >= 3 else 0

# Write output to new TIFF
meta.update(dtype=rasterio.int32)
with rasterio.open(output_tiff, 'w', **meta) as dst:
    dst.write(output_array, 1)

print(f"Output saved to {output_tiff}")


In [ ]:
import rasterio
import os

# Set the directory where your TIFF files are located
tiff_directory = r"/content/drive/MyDrive/Heatstress/2023_0_1"
tiff_files = sorted([f for f in os.listdir(tiff_directory) if f.endswith('.tif')])

# Initialize a dictionary to store the longest streak data for each pixel
# Structure: {pixel_index: {"max_streak": int, "files": [file names]}}
pixel_streaks = {}

# Read the first file to get the dimensions of the raster
with rasterio.open(os.path.join(tiff_directory, tiff_files[0])) as src:
    width, height = src.width, src.height

# Process each pixel position
for y in range(height):
    for x in range(width):
        max_streak = 0
        current_streak = 0
        streak_files = []
        max_streak_files = []

        # Iterate over each file (representing each day)
        for filename in tiff_files:
            filepath = os.path.join(tiff_directory, filename)

            # Open the TIFF file and read the pixel value at (x, y)
            with rasterio.open(filepath) as src:
                pixel_value = src.read(1)[y, x]

            if pixel_value == 1:
                # Increment the current streak and add file to streak_files
                current_streak += 1
                streak_files.append(filename)
            else:
                # If streak ends, check if it was the longest
                if current_streak > max_streak:
                    max_streak = current_streak
                    max_streak_files = streak_files.copy()

                # Reset the current streak
                current_streak = 0
                streak_files = []

        # Final check at the end of the file list
        if current_streak > max_streak:
            max_streak = current_streak
            max_streak_files = streak_files.copy()

        # Store the result if there's any streak of `1`s
        if max_streak > 0:
            pixel_index = (y, x)
            pixel_streaks[pixel_index] = {"max_streak": max_streak, "files": max_streak_files}

# Write the result to a text file
output_file = r"/content/drive/MyDrive/popcons.txt"
with open(output_file, "w") as f:
    for pixel, data in pixel_streaks.items():
        f.write(f"Pixel: {pixel}, Max Streak: {data['max_streak']}, Files: {', '.join(data['files'])}\n")

print(f"Results written to {output_file}")
